In [16]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split 
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

##### Загружаем датасет

In [17]:
df = pd.read_csv('data/insurance.csv')
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


##### Проверяем на наличие пропусков и типы данные 

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


##### Разделение признаков и целевой переменной

In [19]:
x = df.drop(['charges'], axis=1)
y = df['charges'].values

##### Преобразование категориальных признаков в числовые значения и нормализация числовых признаков

In [20]:
x['sex'] = x['sex'].map({'male': 0, 'female': 1})
x['smoker'] = x['smoker'].map({'yes': 1, 'no': 0})
x['region'] = x['region'].map({'southwest': 0, 'southeast': 1, 'northwest': 2, 'northeast': 3})

for feature in ['age', 'bmi', 'children', 'region']:
    x[feature] = (x[feature] - x[feature].mean()) / x[feature].std()
    
x.head()

,age,sex,bmi,children,smoker,region
0,-1.438227,1,-0.453151,-0.908274,1,-1.343402
1,-1.509401,0,0.509431,-0.078738,0,-0.438331
2,-0.797655,0,0.383164,1.580335,0,-0.438331
3,-0.441782,0,-1.305043,-0.908274,0,0.466741
4,-0.512957,0,-0.292447,-0.908274,0,0.466741


##### Выполняем снижение размерности на основе МГК (мин уровень дисперсии 0.95)

In [21]:
pca = PCA(n_components=0.98) # при 0.95 растет полиномиальная линейная модель 
x = pca.fit_transform(x)
x_base = x

##### Выполняем расширение базовых признаков за счет применения полиномиальной модели (степень = 3)

In [22]:
poly = PolynomialFeatures(degree=3, include_bias=False)
x_poly = poly.fit_transform(x_base)

##### Разбиваем датасеты (базовый и расширенный) на 2 множества (обучающее и тестовое).

In [23]:
# Разделение базового набора данных на тренировочные и тестовые
x_base_train, x_base_test, y_base_train, y_base_test = train_test_split(
    x_base, y, test_size=0.3, shuffle=True
)

# Разделение расширенного набора данных на тренировочные и тестовые
x_poly_train, x_poly_test, y_poly_train, y_poly_test = train_test_split(
    x_poly, y, test_size=0.3, shuffle=True
)

##### Выполняем обучение линейной и полиномиальной моделей аналитическим методом - не используя готовое библиотечное решение.


In [24]:
def analytical_solution(x, y):
    x = np.c_[np.ones(x.shape[0]), x] # Добавляем столбец единиц для учета интерсепта
    theta = np.linalg.inv(x.T @ x) @ x.T @ y
    return theta

In [25]:
# Обучение базовой модели
linear_theta_base = analytical_solution(x_base_train, y_base_train)
print("Коэффициенты базовой модели:", linear_theta_base.flatten())

# Обучение расширенной модели
linear_theta_poly = analytical_solution(x_poly_train, y_poly_train)
print("Коэффициенты расширенной модели:", linear_theta_poly.flatten())

Коэффициенты базовой модели: [13273.66644566  2679.07061428  2243.79632889 -2029.37620018
   306.15908485 -4039.93542611 23766.59550228]
Коэффициенты расширенной модели: [ 4.43060249e+18 -1.20577236e+18  1.35454910e+18  5.89567866e+18
 -1.52558077e+16  3.27852073e+17 -7.68218989e+17 -9.70194799e+14
  8.23629103e+15 -2.94223652e+16 -2.86794590e+16  6.78130226e+17
 -8.64576490e+17  2.96290891e+15 -1.83509899e+16 -2.51829506e+15
 -1.83982266e+16 -8.06928790e+17  2.66731752e+16  8.98925039e+15
  1.36498911e+17  3.54464276e+18 -2.39974562e+16  1.31602539e+18
  2.52355791e+17 -1.80259829e+19 -6.60464178e+18  8.13613057e+17
  8.75045525e+14 -2.05674942e+15 -3.08219240e+15  4.00963629e+15
 -1.10691168e+17 -2.80840531e+16  1.30170753e+15 -7.70342097e+13
 -8.62909828e+15  2.33776294e+17  1.87879374e+16 -7.19548210e+15
 -1.71706218e+16  4.76761832e+17  1.46007471e+17  5.04566072e+15
 -2.72431139e+17 -1.29759508e+16  3.69639745e+18  7.07509442e+17
  1.68339605e+18 -1.94100202e+14  1.43240308e+14  

##### Выполняем обучение линейной и полиномиальной моделей градиентным спуском - не используя готовое библиотечное решение.


In [26]:
def gradient_descent(x, y, learning_rate=0.01, iterations=1000):
    m, n = x.shape
    x = np.c_[np.ones(m), x]  # Добавляем столбец единиц для интерсепта
    theta = np.zeros((n + 1,))  # Инициализация коэффициентов
    for i in range(iterations):
        predictions = x @ theta
        error = predictions - y
        theta -= (learning_rate / m) * (x.T @ error)
    return theta

In [27]:
# Обучение базовой модели
gradient_theta_base = gradient_descent(x_base_train, y_base_train, learning_rate=0.0001, iterations=100000)
print("Коэффициенты базовой модели:", gradient_theta_base.flatten())

# Обучение расширенной модели
gradient_theta_poly = gradient_descent(x_poly_train, y_poly_train, learning_rate=0.0001, iterations=100000)
print("Коэффициенты расширенной модели:", gradient_theta_poly.flatten())

Коэффициенты базовой модели: [13273.90033759  2635.5185428   2254.57159429 -1992.80966341
   341.43309367 -3655.49649674 18980.6917166 ]
Коэффициенты расширенной модели: [ 8.75488684e+03  1.76615524e+03  1.49554503e+03 -1.47510937e+03
  3.92600965e+02 -1.31796459e+03  6.46947304e+03  5.13010067e+02
  2.98280884e+02  2.71890500e+01 -6.79183552e+02 -1.24829914e+03
  3.82005905e+03  8.41558269e+02  5.08031338e+02  3.56703042e+02
  1.74855078e+02 -2.08486085e+03  7.95127816e+02  8.94073804e+02
  1.78344560e+02 -1.15167074e+03  1.05186475e+03 -5.59040090e+02
  3.25590227e+03  2.49183581e+03 -1.17643985e+03  4.95096271e+03
  1.08732148e+02  3.85208388e+02  6.38015637e+00 -3.29972934e+02
 -4.82514942e+02  2.38102811e+03  5.19657737e+02 -1.71544533e+01
 -8.65113649e+02  1.53919511e+02 -3.41995426e+01 -1.93573294e+02
  2.01286657e+02 -4.96428885e+02  6.56705969e+02 -7.96181952e+02
 -1.96257638e+02 -1.19498706e+03  5.66023367e+02 -6.82427402e+02
  2.48018804e+03  2.26073561e+01 -4.24878143e+02 -

In [28]:
def predict(x, theta):
    x = np.c_[np.ones(x.shape[0]), x]  # Добавляем столбец единиц для интерсепта, если его нет
    return x @ theta

# Предсказания для базовой модели
y_base_pred_linear = predict(x_base_test, linear_theta_base)
y_base_pred_gradient = predict(x_base_test, gradient_theta_base)

# Предсказания для полиномиальной модели
y_poly_pred_linear = predict(x_poly_test, linear_theta_poly)
y_poly_pred_gradient = predict(x_poly_test, gradient_theta_poly)

print("Метрики для базовой модели (Linear):")
print(f"MAE : {mean_absolute_error(y_base_test, y_base_pred_linear):.3f}")
print(f"MSE : {mean_squared_error(y_base_test, y_base_pred_linear):.3f}")
print(f"R²  : {r2_score(y_base_test, y_base_pred_linear):.3f}")

print("\nМетрики для полиномиальной модели (Linear):")
print(f"MAE : {mean_absolute_error(y_poly_test, y_poly_pred_linear):.3f}")
print(f"MSE : {mean_squared_error(y_poly_test, y_poly_pred_linear):.3f}")
print(f"R²  : {r2_score(y_poly_test, y_poly_pred_linear):.3f}")

print("\nМетрики для базовой модели (Gradient):")
print(f"MAE : {mean_absolute_error(y_base_test, y_base_pred_gradient):.3f}")
print(f"MSE : {mean_squared_error(y_base_test, y_base_pred_gradient):.3f}")
print(f"R²  : {r2_score(y_base_test, y_base_pred_gradient):.3f}")

print("\nМетрики для полиномиальной модели (Gradient):")
print(f"MAE : {mean_absolute_error(y_poly_test, y_poly_pred_gradient):.3f}")
print(f"MSE : {mean_squared_error(y_poly_test, y_poly_pred_gradient):.3f}")
print(f"R²  : {r2_score(y_poly_test, y_poly_pred_gradient):.3f}")

Метрики для базовой модели (Linear):
MAE : 4110.172
MSE : 36471308.391
R²  : 0.745

Метрики для полиномиальной модели (Linear):
MAE : 97425.276
MSE : 10984237772.928
R²  : -64.875

Метрики для базовой модели (Gradient):
MAE : 4536.139
MSE : 38860730.234
R²  : 0.729

Метрики для полиномиальной модели (Gradient):
MAE : 3502.807
MSE : 31297840.393
R²  : 0.812
